In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#            Machine Learning y Deep Learning
# ----------------------------------------------------------
#         Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
# Redes neuronales LSTM y GRU para la prediccion de BitCoin
# ==========================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense

In [ ]:
# ---------------------- Cargando datos ----------------------------------------
# Cargar el conjunto de datos:
url =  "https://raw.githubusercontent.com/rogon666/UMSA/main/AIMLDL/Datos/BTCUSD.csv"
# Cargar los datos en un DataFrame
data= pd.read_csv(url)
# Mostrar las primeras filas del DataFrame
print(data.head())

In [ ]:
# Preparar el objetivo (y)
y = data['Close'].values.reshape(-1, 1)

# Crear la matriz de características X como rezagos de y
def create_lagged_features(y, lag=1):
    X = np.zeros((len(y) - lag, lag))
    for i in range(lag):
        X[:, i] = y[(lag - i - 1):(len(y) - i - 1), 0]
    y_lagged = y[lag:]
    return X, y_lagged

# Usar rezagos para las características
lag = 60
X, y_lagged = create_lagged_features(y, lag)

# Escalar los datos
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X = scaler_X.fit_transform(X)
y_lagged = scaler_y.fit_transform(y_lagged)
'''
Los modelos de redes neuronales, como LSTM y GRU, son sensibles a la escala de los datos.
Si las entradas tienen valores en rangos muy diferentes, los pesos en la red pueden necesitar
ajustes muy diferentes, lo que puede hacer que el entrenamiento sea más difícil y menos estable.
Normalizar los datos asegura que todas las características tengan una magnitud similar,
lo que ayuda a que el modelo aprenda de manera más eficiente.
Cuando los datos están en un rango como [0, 1], el algoritmo de optimización (por ejemplo, descenso de gradiente)
converge más rápidamente porque los gradientes están en un rango más controlado.
Esto ayuda a evitar pasos de actualización demasiado grandes o pequeños durante el entrenamiento,
lo que podría ralentizar la convergencia o hacer que el modelo se atasque en mínimos locales.
'''
# Reshape para que sea compatible con LSTM/GRU (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Partición de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y_lagged, test_size=0.15, shuffle=False)

# Inicialización del modelo LSTM
model_lstm = Sequential()
model_lstm.add(Input(shape=(X_train.shape[1], 1)))  # Definir la capa de entrada explícitamente
model_lstm.add(LSTM(units=50,
                    return_sequences=True, # opcion para pasar las secuencias a la siguiente capa
                    input_shape=(X_train.shape[1], 1)))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dense(1))

# Compilación del modelo LSTM
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo LSTM
epochs = 100
model_lstm.fit(X_train, y_train, epochs=epochs, verbose=0)

# Predicción con el modelo LSTM
y_pred_train_lstm = model_lstm.predict(X_train)
y_pred_test_lstm = model_lstm.predict(X_test)

# Inicialización del modelo GRU
model_gru = Sequential()
model_gru.add(Input(shape=(X_train.shape[1], 1)))  # Definir la capa de entrada explícitamente
model_gru.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_gru.add(GRU(units=50))
model_gru.add(Dense(1))

# Compilación del modelo GRU
model_gru.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo GRU
model_gru.fit(X_train, y_train, epochs=epochs, verbose=0)

# Predicción con el modelo GRU
y_pred_train_gru = model_gru.predict(X_train)
y_pred_test_gru = model_gru.predict(X_test)

# Desnormalizar para evaluar correctamente
y_train_actual = scaler_y.inverse_transform(y_train)
y_test_actual = scaler_y.inverse_transform(y_test)
y_pred_train_lstm_actual = scaler_y.inverse_transform(y_pred_train_lstm)
y_pred_test_lstm_actual = scaler_y.inverse_transform(y_pred_test_lstm)
y_pred_train_gru_actual = scaler_y.inverse_transform(y_pred_train_gru)
y_pred_test_gru_actual = scaler_y.inverse_transform(y_pred_test_gru)

# Evaluación del modelo LSTM
mae_train_lstm = mean_absolute_error(y_train_actual, y_pred_train_lstm_actual)
rmse_train_lstm = np.sqrt(mean_squared_error(y_train_actual, y_pred_train_lstm_actual))
mae_test_lstm = mean_absolute_error(y_test_actual, y_pred_test_lstm_actual)
rmse_test_lstm = np.sqrt(mean_squared_error(y_test_actual, y_pred_test_lstm_actual))

print(f"LSTM MAE (train): {mae_train_lstm}")
print(f"LSTM RMSE (train): {rmse_train_lstm}")
print(f"LSTM MAE (test): {mae_test_lstm}")
print(f"LSTM RMSE (test): {rmse_test_lstm}")

# Evaluación del modelo GRU
mae_train_gru = mean_absolute_error(y_train_actual, y_pred_train_gru_actual)
rmse_train_gru = np.sqrt(mean_squared_error(y_train_actual, y_pred_train_gru_actual))
mae_test_gru = mean_absolute_error(y_test_actual, y_pred_test_gru_actual)
rmse_test_gru = np.sqrt(mean_squared_error(y_test_actual, y_pred_test_gru_actual))

print(f"GRU MAE (train): {mae_train_gru}")
print(f"GRU RMSE (train): {rmse_train_gru}")
print(f"GRU MAE (test): {mae_test_gru}")
print(f"GRU RMSE (test): {rmse_test_gru}")

# Gráficos de comparación
plt.figure(figsize=(14, 7))

# Predicciones vs Realidad para LSTM
plt.subplot(2, 1, 1)
plt.plot(y_test_actual, label='Valores observados')
plt.plot(y_pred_test_lstm_actual, label='Valores predichos LSTM')
plt.title('Comparación entre los precios reales y predichos (LSTM)')
plt.xlabel('Días')
plt.ylabel('Precio Bitcoin (USD)')
plt.legend()

# Predicciones vs Realidad para GRU
plt.subplot(2, 1, 2)
plt.plot(y_test_actual, label='Valores observados')
plt.plot(y_pred_test_gru_actual, label='Valores predichos GRU')
plt.title('Comparación entre los precios reales y predichos (GRU)')
plt.xlabel('Días')
plt.ylabel('Precio Bitcoin (USD)')
plt.legend()

plt.tight_layout()
plt.show()
